In [379]:
%matplotlib inline

In [380]:
import matplotlib.pyplot as plt
#import geoplotlib
from bokeh.io import output_notebook, show
from bokeh.models import WMTSTileSource
from bokeh.tile_providers import OSM, CARTODBPOSITRON, get_provider
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.transform import linear_cmap
from bokeh.palettes import Spectral11
import pandas as pd
import numpy as np
import requests as rq
import math

In [381]:
r = rq.get('https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2010-01-01&endtime=2020-12-31&limit=5000&minmagnitude=6')
raw_data = r.json()

In [382]:
df = pd.json_normalize(raw_data["features"])[["id", "properties.time", "properties.mag", 
                                              "properties.status", "geometry.coordinates"]]

df = pd.concat([df.drop(['geometry.coordinates'], axis=1), df['geometry.coordinates'].apply(pd.Series)], axis=1)
df = df.rename(columns={0: 'lon', 1: 'lat', 2: 'depth', 'properties.time': 'datetime', 
                        'properties.mag': 'mag', 'properties.status': 'status'}, errors='raise')
df['datetime'] = pd.to_datetime(df['datetime'], unit='ms')

df

,id,datetime,mag,status,lon,lat,depth
0,us7000bg4v,2020-09-01 21:09:17.422,6.5,reviewed,-71.3937,-27.9285,14.26
1,us7000bfjx,2020-09-01 04:30:02.029,6.3,reviewed,-71.3058,-28.0391,16.98
2,us7000bfjr,2020-09-01 04:09:28.828,6.8,reviewed,-71.2377,-28.0121,23.00
3,us7000bfbx,2020-08-31 17:24:04.890,6.1,reviewed,70.2388,-3.9946,10.00
4,us7000bf3k,2020-08-30 21:20:29.655,6.5,reviewed,-29.8326,0.7691,10.00
...,...,...,...,...,...,...,...
1578,usp000h5rn,2010-01-05 12:15:32.210,6.8,reviewed,157.5510,-9.0190,15.40
1579,usp000h5ra,2010-01-05 04:55:39.410,6.8,reviewed,-14.6950,-58.1730,13.00
1580,usp000h5np,2010-01-03 22:36:25.640,7.1,reviewed,157.3540,-8.7830,10.00
1581,usp000h5nd,2010-01-03 21:48:02.870,6.6,reviewed,157.4870,-8.7260,10.00


In [383]:
output_notebook()
tile_provider = get_provider(OSM)

Loading BokehJS ...

In [384]:
def wgs84_to_web_mercator(df, lon="lon", lat="lat"):
    """Converts decimal longitude/latitude to Web Mercator format"""
    k = 6378137
    df["x"] = df[lon] * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df

In [385]:
wgs84_to_web_mercator(df)
#size_s = (df['mag'].apply(math.exp)/100).apply(math.floor).copy()
size_s = (df['mag']-5).apply(math.pow, args=(3,)).copy()
df = pd.concat([df, size_s.rename('size')], axis=1)
df

,id,datetime,mag,status,lon,lat,depth,x,y,size
0,us7000bg4v,2020-09-01 21:09:17.422,6.5,reviewed,-71.3937,-27.9285,14.26,-7.947510e+06,-3.239962e+06,3.375
1,us7000bfjx,2020-09-01 04:30:02.029,6.3,reviewed,-71.3058,-28.0391,16.98,-7.937725e+06,-3.253904e+06,2.197
2,us7000bfjr,2020-09-01 04:09:28.828,6.8,reviewed,-71.2377,-28.0121,23.00,-7.930144e+06,-3.250499e+06,5.832
3,us7000bfbx,2020-08-31 17:24:04.890,6.1,reviewed,70.2388,-3.9946,10.00,7.818947e+06,-4.450375e+05,1.331
4,us7000bf3k,2020-08-30 21:20:29.655,6.5,reviewed,-29.8326,0.7691,10.00,-3.320950e+06,8.561839e+04,3.375
...,...,...,...,...,...,...,...,...,...,...
1578,usp000h5rn,2010-01-05 12:15:32.210,6.8,reviewed,157.5510,-9.0190,15.40,1.753850e+07,-1.008163e+06,5.832
1579,usp000h5ra,2010-01-05 04:55:39.410,6.8,reviewed,-14.6950,-58.1730,13.00,-1.635840e+06,-8.003748e+06,5.832
1580,usp000h5np,2010-01-03 22:36:25.640,7.1,reviewed,157.3540,-8.7830,10.00,1.751657e+07,-9.815709e+05,9.261
1581,usp000h5nd,2010-01-03 21:48:02.870,6.6,reviewed,157.4870,-8.7260,10.00,1.753137e+07,-9.751509e+05,4.096


In [386]:
p = figure(tools='pan, wheel_zoom', x_axis_type="mercator", y_axis_type="mercator")

p.add_tile(tile_provider)

mapper = linear_cmap(field_name='size', palette=Spectral11 ,low=min(df['size']) ,high=max(df['size']))
source = ColumnDataSource(data=dict(x=df['x'], y=df['y'], size=df['size']))

p.circle(x='x', y='y', source=source, line_color=mapper, color=mapper, size='size')

show(p)